In [1]:
import pickle
import math
import pandas as pd
import numpy as np
from my_utils import print_red, print_cyan, print_green

"""
Inputs:
df of net load

Outputs:
CFD plot and VRE event df for fingerprint matching
"""

data = pickle.load(open("PickleJar\\netload_components_1980-2019.pickle","rb"))
VRE_profiles = data["VRE_profiles"]
load = data["load"]
cap = data["cap"]
#VRE_profiles
#load
#cap
load_list = []
for year, load in load.items(): 
    load_list += list(load)
load = np.array(load_list)
net_load = -(VRE_profiles*cap).sum(axis=1) + load
net_load

1980  h0001     86.255441
      h0002     88.136690
      h0003     89.460727
      h0004     93.152194
      h0005    100.754412
                  ...    
2019  h8756     88.076780
      h8757     76.828800
      h8758     69.671389
      h8759     66.612360
      h8760     65.359772
Length: 350640, dtype: float64

In [2]:
#net_load=((-1)*df_disp['bat_ch'])
#net_load=(df_disp['WG_peak'])
d = {'net load': net_load,'count1':0,'count2':0}
df_netload = pd.DataFrame(data=d)
#maxind, maxval = max(net_load, key=lambda item: item[1])
maxval=int(math.ceil(net_load.max()))
minval=int(math.floor(net_load.min()))
amps = np.arange(minval, maxval, 0.1).tolist()
print(df_netload, minval,maxval)

              net load  count1  count2
1980 h0001   86.255441       0       0
     h0002   88.136690       0       0
     h0003   89.460727       0       0
     h0004   93.152194       0       0
     h0005  100.754412       0       0
...                ...     ...     ...
2019 h8756   88.076780       0       0
     h8757   76.828800       0       0
     h8758   69.671389       0       0
     h8759   66.612360       0       0
     h8760   65.359772       0       0

[350640 rows x 3 columns] -197 201


In [ ]:
#kolla på diffen av detta
df_freq = pd.DataFrame()
output = {}
dfrows=df_netload.index
#net_loads_array = np.array(df_netload.values)
for amp in amps:
    d = {'net load': net_load,'count1':0,'count2':0}
    df_netload = pd.DataFrame(data=d)
    s=[]
    rowbefore=dfrows[0]
#ökande siffra när värde över amp
    for i_r, row in enumerate(dfrows):
        net_load_val = df_netload.at[row,'net load']
        amp_positive = amp>=0
        amp_negative = amp<0
        netload_greater_than_amp = net_load_val>=amp
        netload_smaller_than_amp = net_load_val<amp
        if amp_positive and netload_greater_than_amp:
#            df_netload.set_value(row, 'count1', df_netload.at[rowbefore,'count1']+1)
            df_netload.at[row, 'count1']= df_netload.at[rowbefore,'count1']+1
        elif amp_negative and netload_smaller_than_amp:
#            df_netload.set_value(row, 'count1', df_netload.at[rowbefore,'count1']+1)
            df_netload.at[row, 'count1']= df_netload.at[rowbefore,'count1']+1
        rowbefore=row
#varje periods längd vid sluttillfället
        if amp >0 and df_netload.at[rowbefore,'net load']>amp and df_netload.at[row,'net load']<amp:
#            df_netload.set_value(rowbefore, 'count2', df_netload.at[rowbefore,'count1'])
            df_netload.at[rowbefore, 'count2']= df_netload.at[rowbefore,'count1']
        elif amp<0 and df_netload.at[rowbefore,'net load']<amp and df_netload.at[row,'net load']>amp:
#            df_netload.set_value(rowbefore, 'count2', df_netload.at[rowbefore,'count1'])
            df_netload.at[rowbefore, 'count2']= df_netload.at[rowbefore,'count1']
        rowbefore=row
        
    s=df_netload.count2.value_counts() 
    df_freq=pd.DataFrame(data=s)
#    s_form=list(s)
    output[amp]=df_freq


#df_out=pd.DataFrame(data=output, index=[amp])
df_out = pd.DataFrame()
df_out_tot = pd.DataFrame()
output2 = {}
for amp in amps:
    df_out = output[amp] 
    df_out = df_out.iloc[1:]
    df_out.index.name = 'Duration'
    df_out = pd.concat([df_out], keys=[amp], names=['Amplitude'])
    df_out.rename(columns={'count2':'Occurences'}, inplace=True)
    df_out_tot=df_out_tot.append(df_out)
#    output2[amp] = df_out
#df_out[0]
df_out_tot

In [ ]:
df_reset = df_out_tot.reset_index()
df_reset.columns = ['Amplitude', 'Duration', 'Occurrence']
df_pivot=df_reset.pivot('Amplitude', 'Duration')

X=df_pivot.columns.levels[1].values
Y=df_pivot.index.values
Z=df_pivot.values
#df1['A'] = df1['A'].apply(lambda x: [y if y <= 9 else 11 for y in x])
Z_BAT=np.where(Z > 50, 50, Z)
X_BAT,Y_BAT = np.meshgrid(X, Y)


In [ ]:
plt.contourf(Ynetload, Xnetload, Znetload, alpha=0.7, cmap=plt.cm.jet);
plt.show()